# အစည်းအဝေး ၁ – Chat Bootstrap (Foundry Local)

ဒီ notebook က Foundry Local ကို bootstrap လုပ်ပြီး၊ အကြိုက်ဆုံး model alias ကို download လုပ်ပြီး၊ သာမန် chat completion နဲ့ streaming chat completion နှစ်ခုလုံးကို လုပ်ဆောင်ပါမယ်။


# အခြေအနေ  
ဒီအစည်းအဝေးမှာ Foundry Local ကို အသုံးပြုပြီး ဒေသခံ အနည်းဆုံး ဘာသာစကားမော်ဒယ်ကို တုံ့ပြန်စေဖို့ အခြေခံအချက်များကို မိတ်ဆက်ပေးပါမည်။ သင်လုပ်ဆောင်ရမည့်အရာများမှာ:  
- SDK / client အခွင့်အရေးများကို ထည့်သွင်းပါ။  
- ရွေးချယ်ထားသော alias (default: `phi-4-mini`) အတွက် Foundry Local manager ကို စတင်ပါ။  
- မော်ဒယ် metadata ထဲမှာ optional fields များကို ခွင့်ပြုနိုင်ရန် အကာအကွယ် monkey‑patch ကို အသုံးပြုပါ။  
- ပုံမှန် chat completion request ကို ပို့ပါ။  
- တုံ့ပြန်မှု token‑by‑token ကို stream လုပ်ပါ။  

ဒီရည်ရွယ်ချက်မှာ RAG, routing, ဒါမှမဟုတ် agents များဆီသို့ ရွှေ့မီ သင့်ဒေသခံ runtime & network path ကို အတည်ပြုရန်ဖြစ်သည်။  


### ရှင်းလင်းချက် - အချိုးအစားများကို ထည့်သွင်းခြင်း
ဤအနည်းဆုံး chat flow အတွက်လိုအပ်သော Python packages များကို ထည့်သွင်းပါသည်။
- `foundry-local-sdk`: ဒေသခံမော်ဒယ်များနှင့် ဝန်ဆောင်မှု၏ အသက်တာကို စီမံရန်။
- `openai`: Chat completions အတွက် ရိုးရှင်းသော client abstraction။
- `rich`: Notebook output ကို ပိုမိုရှင်းလင်းစေရန် pretty printing။

ပြန်လည်လုပ်ဆောင်ခြင်းသည် အန္တိမဖြစ်စေသောကြောင့် လုံခြုံပါသည်။ သင့်ပတ်ဝန်းကျင်တွင် အဆိုပါ packages များရှိပြီးသားဖြစ်ပါက ကျော်သွားနိုင်ပါသည်။


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### ရှင်းလင်းချက် - အဓိက Import များ
Notebook တစ်ခုလုံးတွင် အသုံးပြုသော module များကို ယူဆောင်လာသည်။
- `FoundryLocalManager` သည် ဒေသတွင်း model runtime နှင့် ဆက်သွယ်ရန်။
- `OpenAI` client သည် ကျွန်ုပ်တို့ရင်းနှီးပြီးသား chat completion API surface ကို ပြန်လည်အသုံးပြုနိုင်ရန်။
- `rich.print` သည် အလှဆင် output များအတွက်။

ဒီနေရာမှာ network call မရှိပါ—ဤသည် namespace ကို ပြင်ဆင်ပေးခြင်းသာဖြစ်သည်။


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### ရှင်းလင်းချက်: Manager Initialization & Metadata Patch
`FoundryLocalManager` ကို ရွေးချယ်ထားသော alias အတွက် initialize လုပ်ပြီး `promptTemplate` က `null` ဖြစ်နိုင်သော service response များကို သက်သာစွာ ကိုင်တွယ်နိုင်ရန် defensive monkey-patch ကို အသုံးပြုသည်။

အဓိကရလဒ်များ:
- Service အခြေအနေနှင့် endpoint ကို အတည်ပြုသည်။
- Cached model များကို စာရင်းပြုစုသည် (local store ကို အတည်ပြုသည်)။
- Alias အတွက် concrete model ID ကို ဖြေရှင်းသည် (နောက်ပိုင်း chat calls တွင် အသုံးပြုသည်)။

Raw service metadata တွင် validation ပြဿနာများကို တွေ့ရှိပါက၊ SDK ကို fork မလုပ်ဘဲ sanitize လုပ်နည်းကို ဒီ pattern မှာ ပြသထားသည်။


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### ရှင်းလင်းချက် - အခြေခံ Chat Completion
`OpenAI`-compatible client ကို Foundry ရဲ့ ဒေသခံ endpoint ကိုညွှန်ပြပြီး single non-streaming chat completion ကိုလုပ်ဆောင်ပါသည်။ အဓိကအချက်များမှာ:
- မော်ဒယ်က အမှားမဖြစ်ဘဲ တုံ့ပြန်မှုပေးနိုင်ရန် သေချာပါစေ။
- Latency / output format ကို အတည်ပြုပါ။
- အရင်းအမြစ်များကိုထိန်းသိမ်းရန် `max_tokens` ကို သင့်တော်အောင်ထားပါ။

ဤလုပ်ဆောင်မှုမအောင်မြင်ပါက Foundry Local service ရှိ/မရှိနှင့် alias မှန်ကန်စွာအလုပ်လုပ်နေ/မနေကို ပြန်လည်စစ်ဆေးပါ။


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### ရှင်းလင်းချက်: စီးဆင်းမှု Chat Completion
တိုးတက်သော latency နှင့် အပြန်အလှန် UX အတွက် token streaming ကို ပြသထားသည်။ loop သည် ရောက်ရှိလာသော incremental deltas များကို ပုံနှိပ်ပြသသည်။
- အစပိုင်း output အရေးကြီးသော chat UI များအတွက် အသုံးဝင်သည်။
- token throughput နှင့် full completion latency ကို တိုင်းတာနိုင်စေသည်။

ဤ pattern ကို tokens များစုစည်းခြင်း၊ progress widget ကို update ပြုလုပ်ခြင်း၊ သို့မဟုတ် generation အလယ်တွင် ရပ်တန့်ခြင်းအတွက် အလွယ်တကူ ပြောင်းလဲအသုံးပြုနိုင်သည်။


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရ အရင်းအမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအမှားများ သို့မဟုတ် အနားယူမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
